<a href="https://colab.research.google.com/github/Zamachi/hugging-face-model-finetuning/blob/main/Diplomski_projekat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Uvod

Ovde instaliramo i importujemo neophodne stvari za rad

In [1]:
!pip install transformers # mozda vec ima u /usr/local/lib/python3.7/dist-packages/...
!pip install datasets
!pip install sentencepiece
!pip install accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 4.1 MB/s 
     |████████████████████████████████| 6.6 MB 11.8 MB/s 
     |████████████████████████████████| 101 kB 11.8 MB/s 
     |████████████████████████████████| 596 kB 57.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 4.2 MB/s 
     |████████████████████████████████| 140 kB 19.5 MB/s 
     |████████████████████████████████| 1.1 MB 44.3 MB/s 
     |████████████████████████████████| 212 kB 71.7 MB/s 
     |████████████████████████████████| 127 kB 67.1 MB/s 
     |████████████████████████████████| 271 kB 66.3 MB/s 
     |████████████████████████████████| 144 kB 60.4 MB/s 
     |███████████

In [2]:
import torch
import numpy as np
from transformers.utils.dummy_tokenizers_objects import BertTokenizerFast, DistilBertTokenizerFast, AlbertTokenizerFast, BartTokenizerFast, T5TokenizerFast
from accelerate import Accelerator, notebook_launcher
from transformers import pipeline, AutoTokenizer,AdamW, get_scheduler, BertForSequenceClassification, BertConfig, DistilBertForSequenceClassification, DistilBertConfig, AlbertForSequenceClassification, AlbertConfig, PreTrainedModel, BertForQuestionAnswering, DistilBertForQuestionAnswering, AlbertForQuestionAnswering, BartForConditionalGeneration, BartConfig, T5ForConditionalGeneration, T5Config, AutoModel, DataCollatorWithPadding
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, trainers
from torch.utils.data.dataloader import DataLoader
from datasets import load_dataset, interleave_datasets, load_metric, Features, ClassLabel, Value
from tqdm import tqdm
from collections import defaultdict
from datasets.arrow_dataset import Dataset
from datasets.dataset_dict import DatasetDict
from html import unescape
from transformers.tokenization_utils_fast import PreTrainedTokenizerFast
from torch.optim.optimizer import Optimizer
import unicodedata
from random import randint
from huggingface_hub import notebook_login, Repository, get_full_repo_name

#Razrada

## Ucitavanje datasetova

### Sentiment analysis dataset

In [ ]:
data_sentiment_analysis_stream = load_dataset("tweet_eval", "emotion",streaming=True)

data_sentiment_analysis = defaultdict(list)

features = Features({"label" : ClassLabel(num_classes=4,names=["anger","joy","optimism", "sadness"]), "text" : Value("string")}) # koristimo ovaj objekat da specificiramo Feature-e

for dataset_split, dataset in data_sentiment_analysis_stream.items():
  data_sentiment_analysis_dict = defaultdict(list) # prazan dictionary u koji cemo smestati nase filtrirane podatke
  for row in tqdm(iter(dataset)):
    for key, value in row.items():
      if(value is not None):
        #print(value)
        if(key == "text"):
          data_sentiment_analysis_dict[key].append(unescape(unicodedata.normalize("NFKC", value)))
        elif(key=="label"):
          data_sentiment_analysis_dict[key].append(value)
  data_sentiment_analysis[dataset_split] = Dataset.from_dict(data_sentiment_analysis_dict, features=features) # moramo specificirati features jer ih ne formatiramo lepo ovim pristupom

data_sentiment_analysis = DatasetDict(data_sentiment_analysis)

3257it [00:01, 2356.77it/s]
1421it [00:01, 1037.32it/s]
374it [00:01, 327.68it/s]


{'label': [2, 0, 1, 0, 3, 0, 3, 1, 0, 0],
 'text': ["“Worry is a down payment on a problem you may never have'.  Joyce Meyer.  #motivation #leadership #worry",
  "My roommate: it's okay that we can't spell because we have autocorrect. #terrible #firstworldprobs",
  "No but that's so cute. Atsu was probably shy about photos before but cherry helped her out uwu",
  "Rooneys fucking untouchable isn't he? Been fucking dreadful again, depay has looked decent(ish)tonight",
  "it's pretty depressing when u hit pan on ur favourite highlighter",
  '@user but your pussy was weak from what I heard so stfu up to me bitch . You got to threaten him that your pregnant .',
  'Making that yearly transition from excited and hopeful college returner to sick and exhausted pessimist. #college',
  'Tiller and breezy should do a collab album. Rapping and singing prolly be fire',
  '@user broadband is shocking regretting signing up now #angry #shouldofgonewithvirgin',
  '@user Look at those teef! #growl']}

### Question answering dataset

In [3]:
# data_question_answering_stream = load_dataset("squad_v2", "squad_v2",streaming=True)
data_question_answering = load_dataset("squad_v2", "squad_v2")
data_question_answering = data_question_answering.flatten()
data_question_answering=data_question_answering.rename_column("answers.text", "text")
data_question_answering=data_question_answering.rename_column("answers.answer_start", "start_positions")
data_question_answering=data_question_answering.remove_columns("title")
data_question_answering= data_question_answering.filter(lambda x: x['question'] is not None and x['text'] is not None and x['start_positions'] is not None and x['context'] is not None and len(x['text'])>0 and len(x['start_positions'])>0 and len(x['context'])>0 and x['context'] != "")
data_question_answering

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

Dataset squad_v2 downloaded and prepared to /root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

Parameter 'function'=<function <lambda> at 0x7f828d7610e0> of the transform datasets.arrow_dataset.Dataset.filter@2.0.1 couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/131 [00:00<?, ?ba/s]

  0%|          | 0/12 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'context', 'question', 'text', 'start_positions'],
        num_rows: 86821
    })
    validation: Dataset({
        features: ['id', 'context', 'question', 'text', 'start_positions'],
        num_rows: 5928
    })
})

### Text summarization dataset

In [ ]:
data_text_summarization = load_dataset("cnn_dailymail", "3.0.0")
data_text_summarization

Reusing dataset cnn_dailymail (/root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

### Text transduction dataset

In [ ]:
data_text_translation = load_dataset("wmt16", "de-en")
data_text_translation

Reusing dataset wmt16 (/root/.cache/huggingface/datasets/wmt16/de-en/1.0.0/9e0038fe4cc117bd474d2774032cc133e355146ed0a47021b2040ca9db4645c0)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 4548885
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 2169
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 2999
    })
})

## Tokenizacija - setup

In [4]:
#Emoticons
emoticons =["😍","😂","💕","🔥","😊","😎","✨","💙","😘","📷","🇺🇸","☀","💜","😉","💯","😁","🎄","📸","😜"]

In [5]:
#Encoder-models
#============================================================================================================================
bert_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
distillbert_tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
albert_tokenizer = AutoTokenizer.from_pretrained("albert-base-v2")

bert_tokenizer.add_tokens(emoticons) # https://github.com/huggingface/transformers/issues/7648 see why
distillbert_tokenizer.add_tokens(emoticons)
albert_tokenizer.add_tokens(emoticons)
#Decoder-models
#============================================================================================================================


#Seq2Seq model
#============================================================================================================================


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/684 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/742k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25M [00:00<?, ?B/s]

19

In [6]:
def tokenize_sentence_classification(element):
  result = tokenizer(element["text"])
  result["label"] = torch.nn.functional.one_hot(torch.tensor(element["label"]), num_classes=model.config.num_labels).to(torch.float32).tolist()
  return result

def tokenize_question_answering(element):
  '''
  offset_mapping = [ [(0, 0), (0, 4), (5, 8), (9, 16), (17, 22) ], [...] ] - (0,0) pocetak ili kraj recenice, ostali parovi oznacavaju 
  pocetak i kraj(not inclusive) svakog TOKENA u originalnom stringu(ili question ili context) - npr. ako je token 135 za rec "test3", u recenici bi to bilo "Test1 test2 test3", par bi bio (13,17)

  .sequence_ids(i)
  Koristimo da otkrijemo koji par iz offset_mapping-a pripada question-u, a koji context-u. 0=question, 1=context, None=separator

  start_positions i end_positions (tenzor shape-a (batch_size - vrv 1000 )) Labele koje oznacavaju indeks prvog(x1,y1) i poslednjeg para(x2,y2) iz offset mapiranja, a koristimo za loss computation. 
  Pozicije su fiksi za sequence_length. Pozicije van sekvence nisu uzete u obzir za racunanje loss-a.
  '''
  questions = [question.strip() for question in element['question']]

  tokenized_input = tokenizer(
      questions, 
      element["context"], 
      max_length=None, 
      truncation="only_second", 
      return_offsets_mapping=True,
      return_overflowing_tokens=True, 
      stride=128, 
      # If set to a number along with max_length, the overflowing tokens returned when return_overflowing_tokens=True will contain some tokens from the end of the truncated sequence returned to provide 
      # some overlap between truncated and overflowing sequences. The value of this argument defines the number of overlapping tokens.
      )
  
  offset_mappings = tokenized_input.pop("offset_mapping")
  # Posto neki sample moze imati predugacak kontekst, zbog return_overflowing_tokens=True dobicemo listu feature-a za samo jedan kontekst(ako je kontekst odgovarajuce velicine bice sve normalno)
  # koristicemo ovo da izmapiramo odredjeni feature na odredjeni kontekst
  sample_mapping = tokenized_input.pop("overflow_to_sample_mapping")

  start_positions = [position[0] for position in element["start_positions"]] # ovo radimo da bi flatten-ovali 
  answers = [answer[0] for answer in element["text"]] # ovo radimo da bi flatten-ovali 

  answer_start_positions = []
  answer_end_positions = []
  for index, offset in enumerate(offset_mappings):

    #Za tekucu recenicu; koji par iz offset_mappings-a pripada question-u, a koji context-u
    seq_ids = tokenized_input.sequence_ids(index) 
    # One example can give several spans, this is the index of the example containing this span of text.
    sample_index = sample_mapping[index]

    answer = answers[sample_index] # svaka recenica ima po 1 odgovor. MODIFIED koristimo sample_index jer sada za 1 kontekst/odgovor imamo vise feature-a(ako je kontekst > context_len modela!)
    answer_start_index = start_positions[sample_index] # uzimamo pocetni index tekuce recenice(token-a ?) MODIFIED, bilo je [i], ali tkao sada ne radi, pitanje je da li je ovo dobro?
    answer_end_index = answer_start_index + len(answer)# i idemo do kraja odgovora

    #Na osnovu seq_ids mozemo pronaci gde pocinje kontekst, a gde se zavrsava
    context_indices = [index for index, value in enumerate(seq_ids) if value == 1] # context=1, tako da nam treba prvi i poslednji indeks pojavljivanja jedinice
    context_start = context_indices[0]
    context_end = context_indices[len(context_indices)-1]

    #ako indeksi odgovora(answer_start/end_index) nisu unutar opsega kao i za offset, onda upisujemo 0,0
    #tj. ako je start_index van poslednje vrednosti iz konteksta ili ako je end_index ispod prve vrednosti konteksta, to znaci da odgovor uopste nije u kontekstu(vrv. zbog truncation-a), tada upisujemo par (0,0)
    if answer_start_index > offset[context_end][1] or answer_end_index < offset[context_start][0]:
      answer_start_positions.append(0)
      answer_end_positions.append(0)
    else:
      #u suprotnom znamo da odgovor jeste u kontekstu
      #Za startni indeks pocinjemo od pocetka konteksta i idemo do kraja, ili dok ne naidjemo na pocetni token odgovora(tj. indeks gde se taj token javlja u recenici)
      idx= context_start
      while idx <= context_end and offset[idx][0] <= answer_start_index:
          idx += 1
      answer_start_positions.append(idx - 1)  

      #za end indeks pocinjemo od kraja konteksta i idemo unazad do pocetka ili dok ne naidjemo na krajnji token odgovora(tj. indeks gde se poslednji token iz odgovora javlja u originalnom kontekst tekst formatu)
      idx = context_end
      while idx >= context_start and offset[idx][1] >= answer_end_index:
          idx -= 1
      answer_end_positions.append(idx + 1)
  #NOTE: ako performanse budu lose, blame the function: https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/question_answering.ipynb#scrollTo=tqg1q9oD3_il original ovde
  tokenized_input["start_positions"] = answer_start_positions
  tokenized_input["end_positions"] = answer_end_positions

  return tokenized_input

In [ ]:
#Encoder-models
#============================================================================================================================
#Model configs

albert_config = AlbertConfig()
#============================================================================================================================
#Sentiment analysis

albert_model_sentiment = AlbertForSequenceClassification(albert_config)
#============================================================================================================================
#Extractive question answering
bert_large_model_question_answering = BertForQuestionAnswering(bert_large_config)
distillbert_model_question_answering = DistilBertForSequenceClassification(distillbert_config)
albert_model_question_answering = AlbertForSequenceClassification(albert_config)

#Decoder-model
gpt_neo_125m_model = AutoModel.from_pretrained("EleutherAI/gpt-neo-125M")
gpt_neo_13b_model = AutoModel.from_pretrained("EleutherAI/gpt-neo-1.3B")
gpt2_model = AutoModel.from_pretrained("gpt2")

#Seq2Seq model
#============================================================================================================================
#Model configs
bart_config = BartConfig()
t5_config = T5Config()
#============================================================================================================================
#Both for summarization and transduction
bart_model = BartForConditionalGeneration(bart_config)
t5_model = T5ForConditionalGeneration(t5_config)

Some weights of GPTNeoModel were not initialized from the model checkpoint at EleutherAI/gpt-neo-125M and are newly initialized: ['transformer.h.3.attn.attention.bias', 'transformer.h.7.attn.attention.bias', 'transformer.h.5.attn.attention.bias', 'transformer.h.11.attn.attention.bias', 'transformer.h.1.attn.attention.bias', 'transformer.h.9.attn.attention.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Training 

### Training functions

In [ ]:
def evaluate(model : PreTrainedModel, eval_dataloader:DataLoader, accelerator, metric=load_metric("accuracy"), isCalledFromTraining=False):

    model.eval()
    if(not isCalledFromTraining):
      eval_dataloader = accelerator.prepare( eval_dataloader )

    if("questionanswering" in str(model.__class__.__name__).lower()):
      return evaluate_question_answering(model, eval_dataloader, accelerator) # TODO:
    elif("sequenceclassification" in str(model.__class__.__name__).lower()):
      return evaluate_sentiment(model, eval_dataloader, accelerator)

def evaluate_sentiment(model : PreTrainedModel, eval_dataloader:DataLoader, accelerator, metric=load_metric("accuracy")):
  losses = []
  accuracy = []
  for step, batch in enumerate(eval_dataloader):
      with torch.no_grad():
          outputs = model(batch["input_ids"], labels=batch["labels"]) # ovde je stojalo labels=batch["input_ids"] - ZASTO?! ja sam izmenio na labels
      losses.append(accelerator.gather(outputs.loss))
      metric.add_batch(predictions=torch.argmax(outputs.logits, dim=-1), references=torch.argmax(batch["labels"], dim=-1))
  loss = torch.mean(torch.tensor(losses)) # originalno bilo torch.mean(torch.cat(losses)), sto bi u principu trebalo da uradi konkatenaciju Tenzora i onda da uzme srednju vrednost od toga, ali se ispostavlja da to ne radi
  try:
      perplexity = torch.exp(loss)
  except OverflowError:
      perplexity = float("inf")
  return loss.item(), perplexity.item(), metric.compute()["accuracy"]

def evaluate_question_answering(model : PreTrainedModel, eval_dataloader:DataLoader, accelerator, metric):
  start_logits = []
  end_logits = []
  for batch in enumerate(eval_dataloader):
      with torch.no_grad():
          outputs = model(**batch)
      # start_logits.append(accelerator.gather(outputs.start_logits).cpu().numpy()) # might not be used if on GPU
      start_logits.append(accelerator.gather(outputs.start_logits))
      # end_logits.append(accelerator.gather(outputs.end_logits).cpu().numpy()) # might not be used if on GPU
      end_logits.append(accelerator.gather(outputs.end_logits))
  start_logits = np.concatenate(start_logits)
  end_logits = np.concatenate(end_logits)
  start_logits = start_logits[: len(validation_dataset)]# TODO:
  end_logits = end_logits[: len(validation_dataset)]# TODO:

  formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions.items()]
  references = [{"id": ex["id"], "answers": ex["answers"]} for row in enumerate(eval_dataloader)]
  return metric.compute(predictions=formatted_predictions, references=references)# TODO:
   

def training_procedure(model : PreTrainedModel, train_dataloader : DataLoader, eval_dataloader:DataLoader, output_dir=None, repo:Repository=None, optimizer:Optimizer=torch.optim.AdamW, scheduler="linear", num_warmup_steps=1_000, num_epochs=3, learning_rate=5e-5, gradient_accumulation_steps=8, eval_steps=5_00, metric=load_metric("accuracy")):
  accelerator = Accelerator(fp16=True)

  optimizer = optimizer(model.parameters(), lr=learning_rate)

  train_dl, eval_dl, model, optimizer = accelerator.prepare(
    train_dataloader, eval_dataloader, model, optimizer
  )

  num_training_steps = num_epochs * len(train_dl)

  #optimizer scheduler
  lr_scheduler = get_scheduler(
      scheduler,
      optimizer=optimizer,
      num_warmup_steps=num_warmup_steps,
      num_training_steps=num_training_steps,
  )

  #training loop
  model.train() # turn on training mode
  completed_steps=0
  for epoch in range(num_epochs):
      for step, batch in tqdm( enumerate(train_dl, start=1), total=num_training_steps ):
          outputs = model(**batch)
          loss = outputs.loss 
          if step % 100 == 0:
            accelerator.print(
                {
                    "lr": lr_scheduler.get_lr(),
                    "samples": step * len(batch),
                    "steps": completed_steps,
                    "loss/train": loss.item() * gradient_accumulation_steps,
                }
            )
          loss = loss / gradient_accumulation_steps
          accelerator.backward(loss) # accelerated hugging face backpropagation
          if step % gradient_accumulation_steps == 0:
            accelerator.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            completed_steps += 1
          if (step % (eval_steps * gradient_accumulation_steps)) == 0:
            eval_loss, perplexity, accuracy = evaluate(model,eval_dl, accelerator, metric, isCalledFromTraining=True)
            accelerator.print({"loss/eval": eval_loss, "perplexity": perplexity, "accuracy":accuracy})
            model.train()
            accelerator.wait_for_everyone()
            if output_dir is not None:
              unwrapped_model = accelerator.unwrap_model(model)
              unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
              if accelerator.is_main_process:
                  tokenizer.save_pretrained(output_dir)
                  repo.push_to_hub(
                      commit_message=f"Training in progress step {step}", blocking=False
                  )

Konekcija ka Hugging Face-u da uploadujemo modele

In [7]:
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


### Training Encoders

#### Sentence classification

##### BERT
Inicijalizacija repozitorijuma za ovaj model

In [ ]:
model_name = "bert-base-for-multilabel-sentence-classification"
repo_name = get_full_repo_name(model_name)
repo = Repository(model_name, clone_from=repo_name)

Cloning https://huggingface.co/Zamachi/bert-base-for-multilabel-sentence-classification into local empty directory.


In [ ]:
bert_base_config = BertConfig(vocab_size=len(bert_tokenizer), problem_type="multi_label_classification", num_labels=4)
bert_base_model_sentiment = BertForSequenceClassification(bert_base_config)
# bert_base_model_sentiment.config.num_labels = 4
bert_base_model_sentiment.config.id2label = {
    0: "anger", 1: "joy", 2: "optimism", 3: "sadness"
}
bert_base_model_sentiment.config.label2id = {
    "anger":0, "joy":1,"optimism":2,"sadness":3
}
bert_base_model_sentiment.resize_token_embeddings(len(bert_tokenizer))

data_collator = DataCollatorWithPadding(bert_tokenizer, return_tensors="pt")

model = bert_base_model_sentiment# OVO MORAMO DA URADIMO PRE POZIVA MAPIRANJA
tokenizer = bert_tokenizer # OVO MORAMO DA URADIMO PRE POZIVA MAPIRANJA

tokenized_dataset = data_sentiment_analysis.map(tokenize_sentence_classification, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns("text")
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")
tokenized_dataset.set_format("torch")

train_dataloader = DataLoader(tokenized_dataset["train"], batch_size=16, shuffle=True, collate_fn=data_collator)
eval_dataloader = DataLoader(tokenized_dataset["validation"], batch_size=16, collate_fn=data_collator)
tokenized_dataset["train"][0:1]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

{'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1]]),
 'input_ids': tensor([[  101,  1523,  4737,  2003,  1037,  2091,  7909,  2006,  1037,  3291,
           2017,  2089,  2196,  2031,  1005,  1012, 11830, 11527,  1012,  1001,
          14354,  1001,  4105,  1001,  4737,   102]]),
 'labels': tensor([[0., 0., 1., 0.]]),
 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0]])}

Inference pre treninga

In [ ]:
evaluate(model=bert_base_model_sentiment.to("cuda"), eval_dataloader=eval_dataloader, accelerator=Accelerator(fp16=True))

(0.6784597039222717, 1.9708397388458252, 0.20053475935828877)

In [ ]:
args= (model, train_dataloader, eval_dataloader, repo_name, repo, torch.optim.AdamW, "linear", 1_500, 50, 8.3913e-06, 8, 5_00, load_metric("accuracy"))
notebook_launcher(training_procedure, args=args, use_fp16=True)

Launching training on one GPU.


  1%|          | 97/10200 [00:05<08:50, 19.06it/s]/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:249: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "
  1%|          | 103/10200 [00:05<08:26, 19.94it/s]

{'lr': [6.71304e-08], 'samples': 400, 'steps': 12, 'loss/train': 1.030895471572876}


  2%|▏         | 200/10200 [00:10<09:30, 17.52it/s]

{'lr': [1.342608e-07], 'samples': 800, 'steps': 24, 'loss/train': 2.0408835411071777}


...

  1%|          | 102/10200 [00:05<08:26, 19.93it/s]

{'lr': [6.9200254000000005e-06], 'samples': 400, 'steps': 1237, 'loss/train': 0.29165053367614746}


  2%|▏         | 200/10200 [00:10<09:17, 17.93it/s]

{'lr': [6.9871558e-06], 'samples': 800, 'steps': 1249, 'loss/train': 0.1115926206111908}


  2%|▏         | 204/10200 [00:10<08:49, 18.86it/s]


Inference nakon treninga

In [ ]:
evaluate(model=bert_base_model_sentiment, eval_dataloader=eval_dataloader, accelerator=Accelerator(fp16=True))

(0.7291440963745117, 2.073305368423462, 0.5401069518716578)

##### BERT Large

Inicijalizacija repozitorijuma za ovaj model

In [ ]:
model_name = "bert-large-for-multilabel-sentence-classification"
repo_name = get_full_repo_name(model_name)
repo = Repository(model_name, clone_from=repo_name)

Cloning https://huggingface.co/Zamachi/bert-large-for-multilabel-sentence-classification into local empty directory.


In [ ]:
id2label = {
    0: "anger", 1: "joy", 2: "optimism", 3: "sadness"
}
label2id = {
    "anger":0, "joy":1,"optimism":2,"sadness":3
}
bert_large_config = BertConfig(vocab_size=len(bert_tokenizer), problem_type="multi_label_classification", num_labels=4, id2label=id2label, label2id=label2id, hidden_size=1024,num_hidden_layers=24,num_attention_heads=16)
bert_large_model_sentiment = BertForSequenceClassification(bert_large_config)
bert_large_model_sentiment.resize_token_embeddings(len(bert_tokenizer))

data_collator = DataCollatorWithPadding(bert_tokenizer, return_tensors="pt")

model = bert_large_model_sentiment# OVO MORAMO DA URADIMO PRE POZIVA MAPIRANJA
tokenizer = bert_tokenizer # OVO MORAMO DA URADIMO PRE POZIVA MAPIRANJA

tokenized_dataset = data_sentiment_analysis.map(tokenize_sentence_classification, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns("text")
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")
tokenized_dataset.set_format("torch")

train_dataloader = DataLoader(tokenized_dataset["train"], batch_size=16, shuffle=True, collate_fn=data_collator)
eval_dataloader = DataLoader(tokenized_dataset["validation"], batch_size=16, collate_fn=data_collator)
tokenized_dataset["train"][0:1]

Parameter 'function'=<function tokenize at 0x7fde10319b90> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

{'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1]]),
 'input_ids': tensor([[  101,  1523,  4737,  2003,  1037,  2091,  7909,  2006,  1037,  3291,
           2017,  2089,  2196,  2031,  1005,  1012, 11830, 11527,  1012,  1001,
          14354,  1001,  4105,  1001,  4737,   102]]),
 'labels': tensor([[0., 0., 1., 0.]]),
 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0]])}

Inference pre treninga

In [ ]:
evaluate(model=bert_large_model_sentiment, eval_dataloader=eval_dataloader, accelerator=Accelerator(fp16=True))

(0.6438909769058228, 1.903874397277832, 0.23796791443850268)

Training

In [ ]:
args= (model, train_dataloader, eval_dataloader, repo_name, repo, torch.optim.AdamW, "linear", 1_000, 10, 5e-5, 8, 5_00, load_metric("accuracy"))
notebook_launcher(training_procedure, args=args, use_fp16=True)

Launching training on one GPU.


  5%|▍         | 99/2040 [00:13<04:12,  7.69it/s]/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:249: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "
  5%|▍         | 101/2040 [00:13<03:54,  8.26it/s]

{'lr': [6.000000000000001e-07], 'samples': 400, 'steps': 12, 'loss/train': 4.143622398376465}


 10%|▉         | 199/2040 [00:26<03:50,  7.98it/s]

{'lr': [1.2000000000000002e-06], 'samples': 800, 'steps': 24, 'loss/train': 4.086145401000977}


...

  5%|▍         | 101/2040 [00:16<04:29,  7.20it/s]

{'lr': [1.185e-05], 'samples': 400, 'steps': 237, 'loss/train': 2.0559775829315186}


 10%|▉         | 199/2040 [00:29<03:35,  8.54it/s]

{'lr': [1.2450000000000001e-05], 'samples': 800, 'steps': 249, 'loss/train': 2.092057228088379}


 10%|█         | 204/2040 [00:30<04:36,  6.64it/s]


Inference nakon treninga

In [ ]:
evaluate(model=bert_large_model_sentiment, eval_dataloader=eval_dataloader, accelerator=Accelerator(fp16=True))

(0.5708703398704529, 1.7698067426681519, 0.39037433155080214)

##### Distillbert

Inicijalizacija repozitorijuma za ovaj model

In [ ]:
model_name = "distillbert-for-multilabel-sentence-classification"
repo_name = get_full_repo_name(model_name)
repo = Repository(model_name, clone_from=repo_name)

Cloning https://huggingface.co/Zamachi/distillbert-for-multilabel-sentence-classification into local empty directory.


In [ ]:
id2label = {
    0: "anger", 1: "joy", 2: "optimism", 3: "sadness"
}
label2id = {
    "anger":0, "joy":1,"optimism":2,"sadness":3
}
distillbert_config = DistilBertConfig(vocab_size=len(distillbert_tokenizer), problem_type="multi_label_classification", num_labels=4, id2label=id2label, label2id=label2id)
distillbert_model_sentiment = DistilBertForSequenceClassification(distillbert_config)
distillbert_model_sentiment.resize_token_embeddings(len(distillbert_tokenizer))

data_collator = DataCollatorWithPadding(distillbert_tokenizer, return_tensors="pt")

model = distillbert_model_sentiment# OVO MORAMO DA URADIMO PRE POZIVA MAPIRANJA
tokenizer = distillbert_tokenizer # OVO MORAMO DA URADIMO PRE POZIVA MAPIRANJA

tokenized_dataset = data_sentiment_analysis.map(tokenize_sentence_classification, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns("text")
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")
tokenized_dataset.set_format("torch")

train_dataloader = DataLoader(tokenized_dataset["train"], batch_size=16, shuffle=True, collate_fn=data_collator)
eval_dataloader = DataLoader(tokenized_dataset["validation"], batch_size=16, collate_fn=data_collator)
tokenized_dataset["train"][0:1]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

{'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1]]),
 'input_ids': tensor([[  101,  1523,  4737,  2003,  1037,  2091,  7909,  2006,  1037,  3291,
           2017,  2089,  2196,  2031,  1005,  1012, 11830, 11527,  1012,  1001,
          14354,  1001,  4105,  1001,  4737,   102]]),
 'labels': tensor([[0., 0., 1., 0.]])}

Inference pre treninga

In [ ]:
evaluate(model=model.to("cuda:0"), eval_dataloader=eval_dataloader, accelerator=Accelerator(fp16=True))

(0.6903775334358215, 1.994468331336975, 0.42513368983957217)

Training

In [ ]:
args= (model, train_dataloader, eval_dataloader, repo_name, repo, torch.optim.AdamW, "linear", 1_000, 10, 5e-5, 8, 5_00, load_metric("accuracy"))
notebook_launcher(training_procedure, args=args, use_fp16=True)

Launching training on one GPU.


  5%|▍         | 98/2040 [00:03<01:00, 31.99it/s]/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:249: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "
  5%|▌         | 102/2040 [00:03<00:59, 32.74it/s]

{'lr': [6.000000000000001e-07], 'samples': 300, 'steps': 12, 'loss/train': 5.445560455322266}


 10%|█         | 204/2040 [00:06<00:58, 31.59it/s]


{'lr': [1.2000000000000002e-06], 'samples': 600, 'steps': 24, 'loss/train': 5.146114349365234}


...

{'lr': [1.1200000000000001e-05], 'samples': 600, 'steps': 224, 'loss/train': 3.999934196472168}


  5%|▌         | 105/2040 [00:03<01:01, 31.45it/s]

{'lr': [1.185e-05], 'samples': 300, 'steps': 237, 'loss/train': 4.074880123138428}


 10%|█         | 204/2040 [00:06<00:56, 32.30it/s]

{'lr': [1.2450000000000001e-05], 'samples': 600, 'steps': 249, 'loss/train': 3.9244112968444824}


Inference nakon treninga

In [ ]:
evaluate(model=model.to("cuda"), eval_dataloader=eval_dataloader, accelerator=Accelerator(fp16=True))

(0.48368382453918457, 1.622038722038269, 0.49732620320855614)

##### Albert

Inicijalizacija repozitorijuma za ovaj model

In [ ]:
model_name = "albert-for-multilabel-sentence-classification"
repo_name = get_full_repo_name(model_name)
repo = Repository(model_name, clone_from=repo_name)

Cloning https://huggingface.co/Zamachi/albert-for-multilabel-sentence-classification into local empty directory.


In [ ]:
id2label = {
    0: "anger", 1: "joy", 2: "optimism", 3: "sadness"
}
label2id = {
    "anger":0, "joy":1,"optimism":2,"sadness":3
}
albert_config = AlbertConfig(vocab_size=len(albert_tokenizer), problem_type="multi_label_classification", num_labels=4, id2label=id2label, label2id=label2id)
albert_model_sentiment = AlbertForSequenceClassification(albert_config)
albert_model_sentiment.resize_token_embeddings(len(albert_tokenizer))

data_collator = DataCollatorWithPadding(albert_tokenizer, return_tensors="pt")

model = albert_model_sentiment# OVO MORAMO DA URADIMO PRE POZIVA MAPIRANJA
tokenizer = albert_tokenizer # OVO MORAMO DA URADIMO PRE POZIVA MAPIRANJA

tokenized_dataset = data_sentiment_analysis.map(tokenize_sentence_classification, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns("text")
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")
tokenized_dataset.set_format("torch")

train_dataloader = DataLoader(tokenized_dataset["train"], batch_size=16, shuffle=True, collate_fn=data_collator)
eval_dataloader = DataLoader(tokenized_dataset["validation"], batch_size=16, collate_fn=data_collator)
tokenized_dataset["train"][0:1]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

{'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1]]),
 'input_ids': tensor([[    2,    13,     1, 10041,   622,    25,    21,   125,  7582,    27,
             21,  1448,    42,   123,   243,    57,    22,     9, 12675, 10078,
              9,  6926, 24271,   857,  6926, 15689,  2418,  6926, 10041,   622,
              3]]),
 'labels': tensor([[0., 0., 1., 0.]]),
 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0]])}

Inference pre treninga

In [ ]:
evaluate(model=model.to("cuda"), eval_dataloader=eval_dataloader, accelerator=Accelerator(fp16=True))

(0.8384177088737488, 2.3127048015594482, 0.23796791443850268)

Training

In [ ]:
args= (model, train_dataloader, eval_dataloader, repo_name, repo, torch.optim.AdamW, "linear", 1_000, 10, 5e-5, 8, 5_00, load_metric("accuracy"))
notebook_launcher(training_procedure, args=args, use_fp16=True)

Launching training on one GPU.


  5%|▍         | 99/2040 [01:12<21:30,  1.50it/s]/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:249: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "
  5%|▍         | 100/2040 [01:12<22:08,  1.46it/s]

{'lr': [6.000000000000001e-07], 'samples': 400, 'steps': 12, 'loss/train': 4.012533187866211}


 10%|▉         | 199/2040 [02:24<20:35,  1.49it/s]

{'lr': [1.2000000000000002e-06], 'samples': 800, 'steps': 24, 'loss/train': 3.397462844848633}


...

  5%|▍         | 100/2040 [01:12<24:30,  1.32it/s]

{'lr': [1.185e-05], 'samples': 400, 'steps': 237, 'loss/train': 3.2226076126098633}


 10%|▉         | 199/2040 [02:24<21:44,  1.41it/s]

{'lr': [1.2450000000000001e-05], 'samples': 800, 'steps': 249, 'loss/train': 3.6700315475463867}


 10%|█         | 204/2040 [02:27<22:09,  1.38it/s]


Inference nakon treninga

In [ ]:
evaluate(model=model.to("cuda"), eval_dataloader=eval_dataloader, accelerator=Accelerator(fp16=True))

(0.628760039806366, 1.8752838373184204, 0.4732620320855615)

#### Question answering

##### BERT

Inicijalizacija repozitorijuma za ovaj model

In [ ]:
model_name = "bert-base-for-question-answering"
repo_name = get_full_repo_name(model_name)
repo = Repository(model_name, clone_from=repo_name)

Cloning https://huggingface.co/Zamachi/bert-base-for-question-answering into local empty directory.


In [7]:
bert_base_config = BertConfig(vocab_size=len(bert_tokenizer))
bert_base_model_question_answering = BertForQuestionAnswering(bert_base_config)
bert_base_model_question_answering.resize_token_embeddings(len(bert_tokenizer))

data_collator = DataCollatorWithPadding(bert_tokenizer, padding="max_length", return_tensors="pt")

model = bert_base_model_question_answering# OVO MORAMO DA URADIMO PRE POZIVA MAPIRANJA
tokenizer = bert_tokenizer # OVO MORAMO DA URADIMO PRE POZIVA MAPIRANJA

tokenized_dataset = data_question_answering.map(tokenize_question_answering, batched=True, remove_columns=data_question_answering["train"].column_names)
tokenized_dataset.set_format("torch")

num_rows_in_val=data_question_answering["validation"].num_rows
desired_no=16

train_dataloader = DataLoader(tokenized_dataset["train"], batch_size=desired_no, shuffle=True, collate_fn=data_collator)
eval_dataloader = DataLoader(tokenized_dataset["validation"], batch_size=[i for i in range(desired_no,num_rows_in_val) if num_rows_in_val % i == 0][0], collate_fn=data_collator)
tokenized_dataset["train"][0:1]

  0%|          | 0/87 [00:00<?, ?ba/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

{'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1]]),
 'end_positions': tensor([78]),
 'input_ids': tensor([[  101,  2043,  2106, 20773,  2707,  3352,  2759,  1029,   102, 20773,
          21025, 19358, 22815,  1011,  5708,  1006,  1013, 12170, 23432, 29715,
           3501, 29678, 12325, 29685,  1013, 10506,  1011, 10930,  2078,  1011,
           2360,  1007,  1006,  2141,  2244,  1018,  1010,  3261,  1007,  2003,
           2

In [8]:
model=bert_base_model_question_answering.to("cuda")
model.eval()
accelerator=Accelerator(fp16=True)
eval_dataloader = accelerator.prepare( eval_dataloader )
start_logits = []
end_logits = []
metric=load_metric("squad_v2")
for step, batch in enumerate(eval_dataloader):
  with torch.no_grad():
      outputs = model(**batch)
  # start_logits.append(accelerator.gather(outputs.start_logits).cpu().numpy()) # might not be used if on GPU
  start_logits.append(accelerator.gather(outputs.start_logits))
  # end_logits.append(accelerator.gather(outputs.end_logits).cpu().numpy()) # might not be used if on GPU
  end_logits.append(accelerator.gather(outputs.end_logits))
duzina_starta_pre = len(start_logits)
duzina_enda_pre = len(end_logits)
print(start_logits)
start_logits = torch.cat(start_logits)
end_logits = torch.cat(end_logits)
duzina_starta_posle = len(start_logits)
duzina_enda_posle = len(end_logits)

# The metric takes two files or two lists - one representing model predictions and the other the references to compare them to.

# Predictions : List of triple for question-answers to score with the following key-value pairs:

# 'id': the question-answer identification field of the question and answer pair
# 'prediction_text' : the text of the answer
# 'no_answer_probability' : the probability that the question has no answer


# References: List of question-answers dictionaries with the following key-value pairs:

# 'id': id of the question-answer pair (see above),
# 'answers': a list of Dict {‘text’: text of the answer as a string}
# 'no_answer_threshold': the probability threshold to decide that a question has no answer.


# start_logits = start_logits[: len(validation_dataset)]# TODO:
# end_logits = end_logits[: len(validation_dataset)]# TODO:
references = [{"id": row["id"], "answers": [{"text":text} for index, text in enumerate(row["text"])] , "no_answer_threshold" : 0} for index,row in enumerate(data_question_answering["validation"])]
# return metric.compute(predictions=formatted_predictions, references=references)# TODO:

RuntimeError: ignored

In [23]:
answer_start_index = outputs.start_logits.argmax()
answer_end_index = outputs.end_logits.argmax()
print(answer_start_index)
print(answer_end_index)
predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
# tokenizer.decode(predict_answer_tokens)

tensor(706, device='cuda:0')
tensor(1547, device='cuda:0')


In [14]:
torch.argsort(start_logits, descending=True) # sortira niz i onda nam vraca listu indeksa vrednosti iz originalnog niza(u opadajucem redosledu, ali prvih 20)

tensor([[194, 210, 237,  ...,  67,  26,  29],
        [194, 210, 237,  ...,  19,  26,  67],
        [194,   1, 210,  ...,  19,  26,  67],
        ...,
        [210, 237, 335,  ...,  96, 145,  54],
        [237, 210, 335,  ...,  67,  83, 192],
        [  5, 237, 210,  ...,  96, 145,  54]], device='cuda:0')

In [19]:
len(torch.argsort(start_logits))

5974

In [18]:
len(np.argsort(start_logits.cpu().numpy())[-1 : -20 - 1 : -1].tolist())

20

Inference pre treninga

In [ ]:
evaluate(model=bert_base_model_question_answering.to("cuda"), eval_dataloader=eval_dataloader, accelerator=Accelerator(fp16=True), metric=load_metric("squad_v2"))

Trening

In [ ]:
args= (model, train_dataloader, eval_dataloader, repo_name, repo, torch.optim.AdamW, "linear", 1_500, 50, 8.3913e-06, 8, 5_00, load_metric("squad_v2"))
notebook_launcher(training_procedure, args=args, use_fp16=True)

Inference nakon treninga

In [ ]:
evaluate(model=bert_base_model_sentiment, eval_dataloader=eval_dataloader, accelerator=Accelerator(fp16=True), metric=load_metric("squad_v2"))

##### BERT Large

##### DistillBERT

##### Albert